In [1]:
import pandas as pd
import os
import json
import requests
from bs4 import BeautifulSoup
import re
import random
import numpy as np
import os

c:\Users\Bilal\.conda\envs\textMining\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Bilal\.conda\envs\textMining\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\Bilal\.conda\envs\textMining\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Define the base URL for the Premier League players page
base_url = "https://www.worldfootball.net/players_list/eng-premier-league-2023-2024/nach-name/"

In [3]:
# Create a list to store all players
all_players = []

In [4]:
# Loop through the URLs and send HTTP GET requests
for i in range(1, 13):
    url = base_url + str(i) + "/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find('table', class_='standard_tabelle')
        
        if table:
            # Find all the table rows (tr) within the table with class 'standard_tabelle'
            rows = table.find_all('tr')
            
            # Loop through the rows and extract player names
            for row in rows:
                # Check if the row contains a link (a) with player information
                player_link = row.find('a', href=True)
                
                if player_link:
                    # Extract the player name and add it to the list
                    player_name = player_link.get_text()
                    all_players.append(player_name)

In [5]:
#convert all strings to lower case
all_players = list(map(lambda x: x.lower(), all_players))
#remove whitespace in front and after the names
all_players = list(map(lambda x: x.strip(), all_players))

In [6]:
print(len(all_players))

600


In [7]:
#directories containing wikipedia articles
dirA = r'C:\Users\Bilal\Desktop\TUe\Y2\Q1\TM\enwiki20230820-stripped-json\enwiki20230820-stripped-json\AA'
dirB = r'C:\Users\Bilal\Desktop\TUe\Y2\Q1\TM\enwiki20230820-stripped-json\enwiki20230820-stripped-json\AB'

In [8]:
#initiate list that will contain the wikipedia articles about the players
player_articles = []

In [9]:
#iterate over all files and selecting the lines whereby the title equals the name of the football players
for dir in [dirA, dirB]:
    for file in os.listdir(dir):
        with open(dir+'\\'+file, 'r') as fp:
            while True:
                line = fp.readline()
                if line == '':
                    fp.close()
                    break
                line = json.loads(line)
                title = line['title'].lower().strip()
                if title in all_players:
                    player_articles.append(line)

In [10]:
len(player_articles)

572

In [11]:
#select only articles which are not empty
player_articles = list(filter(lambda x: x['text'] != '', player_articles))

In [12]:
len(player_articles)

512

In [13]:
#select only articles if they contain football or soccer in the text
player_articles = list(filter(lambda x: 'footballer' in x['text'] or 'soccer player' in x['text'], player_articles))

In [14]:
len(player_articles)

438

In [15]:
already_used = [i[:-4] for i in os.listdir(r'football_articles')]

In [16]:
len(already_used)

111

In [17]:
player_articles = list(filter(lambda x: x['title'] not in already_used, player_articles))

In [18]:
len(player_articles)

346

In [19]:
count = 0
index = 0
while count <= 35:
    title = player_articles[index]['title']
    n_words = player_articles[index]['text'].count(' ')
    if n_words > 50:
        with open(r'to_be_annotated\\test\\'+title+'.txt', 'w') as fp:
            text = player_articles[index]['text'].split(' ')[:min([200, n_words])]
            new_text = ' '.join(text)
            new_text = new_text.replace('\n', ' ')
            fp.write(new_text)
            fp.close()
            count += 1
    index += 1

In [25]:
count = 0
index = 0
while count <= 175:
    title = player_articles[index]['title']
    if title+'.txt' in os.listdir(r'to_be_annotated\\test'):
        index += 1
        continue
    else:
        n_words = player_articles[index]['text'].count(' ')
        if n_words > 50:
            with open(r'to_be_annotated\\train\\'+title+'.txt', 'w') as fp:
                text = player_articles[index]['text'].split(' ')[:min([200, n_words])]
                new_text = ' '.join(text)
                new_text = new_text.replace('\n', ' ')
                fp.write(new_text)
                fp.close()
        count += 1
        index += 1